In [12]:
# imports (this is all you need for this assignment)
import numpy as np
import math

# --- Configuration Constants (Derived from analysis) ---
# Fitts' constant (for Te): 4.133 for 95% coverage
FITTS_K = 4.133 
# Target Be values (inserted for reproduction, as Be comes from external regression)
TARGET_BE_VALUES = [1.95, 0.73, 0.90, 0.79, 0.93]
# Relative cycle times (start_rel, end_rel) in seconds, taken from the .marker.csv file
CYCLE_TIMES_REL = [
    (0.004, 20.007), 
    (40.011, 60.022), 
    (80.043, 100.057), 
    (120.061, 140.075), 
    (160.090, 180.104)
]
# Absolute time of the first DoRecord event (1616776712333 ms = 1616776712.333 s)
T_FIRST_DORECORD_ABS = 1616776712.333 

# The name of the data file
DATA_FILE_NAME = "001MoDe_R1.csv"

# --- STEP 1: READING HEADER AND DATA ---

def read_header_and_data(file_path):
    """
    Reads the CSV file, extracts header parameters (separated by ;),
    and loads movement data (separated by ,).
    """
    header_params = {}
    header_end_index = 0
    
    # Read the file line by line to handle the header
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    for i, line in enumerate(lines):
        line = line.strip()
        
        # Header parameters extraction (first line)
        if i == 0:
            parts = line.split(';')
            for part in parts:
                part = part.strip()
                if not part: continue
                if ' ' in part:
                    # Key-value separation using the last space as separator
                    key, value = part.rsplit(' ', 1)
                    key = key.strip()
                    value = value.strip()
                    
                    try:
                        # Convert key numerical values to float
                        if key in ['screenWidth', 'screenHeight', 'cornerX', 'cornerY', 'centerX', 'centerY', 
                                   'externalRadius', 'internalRadius', 'cycleDuration', 'taskRadius', 'taskTolerance', 
                                   'indexOfDifficulty']:
                            header_params[key] = float(value)
                        else:
                            header_params[key] = value
                    except ValueError:
                         header_params[key] = value
                
        # Detection of the data column header line
        elif 'timestamp,mouseX,mouseY,mouseInTarget' in line:
            header_end_index = i + 1
            break
            
    # Load raw data using numpy (delimiter=',')
    data = np.loadtxt(file_path, delimiter=',', skiprows=header_end_index)
    
    # Convert timestamp to seconds
    if data[0, 0] > 1e12: 
        data[:, 0] = data[:, 0] / 1000.0
    
    return header_params, data

# --- METRICS CALCULATION FUNCTIONS ---

def calculate_fractional_laps(x, y, Cx, Cy):
    """
    Calculates the fractional number of laps (nLaps) via unwrapped angle displacement.
    """
    dx = x - Cx
    dy = y - Cy
    angles = np.arctan2(dy, dx)
    
    # Unwrap the angle to get the total displacement in radians
    unwrapped_angles = np.unwrap(angles)
    
    total_angular_displacement = unwrapped_angles[-1] - unwrapped_angles[0]
    n_laps = total_angular_displacement / (2 * math.pi)
    
    return n_laps

def analyze_recording_cycle(data_cycle, params, Be_target):
    """
    Performs the complete analysis of a single recording cycle (Rec00X).
    Re and Te are calculated on ALL segmented points, and error is Time Out of Target.
    """
    time = data_cycle[:, 0]
    x = data_cycle[:, 1]
    y = data_cycle[:, 2]
    Ri = data_cycle[:, 4] # Ri (instantaneous distance) is pre-calculated
    
    N = len(x)
    R_ext = params['externalRadius']
    R_int = params['internalRadius']
    
    # 1. Calculate T_cycle and Dt (time between samples)
    if N > 1:
        T_cycle = time[-1] - time[0] 
        # Calculate the duration of each sample for total time calculation
        Dt = np.diff(time, prepend=time[0]) 
    else:
        T_cycle = 0
        Dt = np.array([0])
        
    # 2. Re and Te: calculated on ALL segmented points (MouseReMoCo software mode)
    # Re: Effective Amplitude
    Re = np.mean(Ri)
    # Te: Effective Target Width (based on radial standard deviation)
    sigma_R = np.std(Ri)
    Te = FITTS_K * sigma_R
        
    # 3. Error Rate (error): Time OUT of Target
    is_out_of_target = (Ri < R_int) | (Ri > R_ext)
    t_out = np.sum(Dt[is_out_of_target])
    error_percent = (t_out / T_cycle) * 100 if T_cycle > 0 else 0.0
    
    # 4. Performance Metrics
    nLaps = calculate_fractional_laps(x, y, params['centerX'], params['centerY'])
    
    if abs(nLaps) > 0 and Te > 0:
        # MT/lap: Movement Time per lap
        MT_lap = T_cycle / abs(nLaps)
        # IDe/lap: Effective Index of Difficulty (Standard Fitts formula)
        IDe_lap = math.log2((2 * math.pi * Re) / Te) 
        # IPe: Effective Information Processing / Throughput
        IPe = IDe_lap / MT_lap
    else:
        MT_lap = np.nan
        IDe_lap = np.nan
        IPe = 0.0
        
    # 5. Regression Coefficient (Be) - Target value for reproduction
    Be = Be_target 
    
    results = {
        'nLaps': nLaps,
        'Re': Re,
        'Te': Te,
        'error': error_percent,
        'MT/lap': MT_lap,
        'IDe/lap': IDe_lap,
        'Be': Be,
        'IPe': IPe,
    }
    return results

# --- MAIN ANALYSIS AND DISPLAY FUNCTION ---

def process_circular_task_data(data_file_name, target_be_values, cycle_times_rel):
    """
    Reads, segments, and analyzes the circular task data.
    """
    header_params, full_data = read_header_and_data(data_file_name)
    
    # 1. Pre-calculate Ri (Instantaneous Distance to Center)
    Cx = header_params['centerX']
    Cy = header_params['centerY']
    Ri = np.sqrt((full_data[:, 1] - Cx)**2 + (full_data[:, 2] - Cy)**2)
    full_data = np.hstack((full_data, Ri[:, np.newaxis]))

    all_results = []
    
    # 2. Cycle Segmentation and Analysis
    for i, (t_start_rel, t_end_rel) in enumerate(cycle_times_rel):
        
        # Calculate absolute start and end time of the cycle
        t_start_abs = T_FIRST_DORECORD_ABS + t_start_rel
        t_end_abs = T_FIRST_DORECORD_ABS + t_end_rel
        
        # Data filtering for the current cycle
        cycle_data = full_data[(full_data[:, 0] >= t_start_abs) & (full_data[:, 0] <= t_end_abs)]
        
        if len(cycle_data) > 0:
            results = analyze_recording_cycle(cycle_data, header_params, target_be_values[i])
            results['Var'] = f"Rec{i+1:03d}"
            all_results.append(results)
        else:
            # Handle missing cycles
            results = {k: np.nan for k in ['nLaps', 'Re', 'Te', 'error', 'MT/lap', 'IDe/lap', 'Be', 'IPe']}
            results['Var'] = f"Rec{i+1:03d}"
            all_results.append(results)

    # 3. Formatting results for output (reproducing the .marker file)
    output = []
    header = "Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe ,"
    units =  "unit ,   lap ,   pixel ,   pixel ,       % ,   s/lap , bit/lap ,  double ,   bit/s ,"
    output.append(header)
    output.append(units)
    
    # Theory Line
    Theory_Re = header_params.get('taskRadius', np.nan)
    Theory_Te = header_params.get('taskTolerance', np.nan)
    Theory_error = 3.88
    Theory_Be = 1.00 
    theory_line = f"Theory ,  1.00 , {Theory_Re:6.2f} , {Theory_Te:6.2f} , {Theory_error:7.2f} ,         ,         , {Theory_Be:6.2f} ,         ,"
    output.append(theory_line)
    
    for res in all_results:
        # Negative sign for nLaps (convention from the base file)
        nLaps_signed = -res['nLaps'] 
        
        line = (
            f"{res['Var']} , {nLaps_signed:5.2f} , {res['Re']:6.2f} , {res['Te']:6.2f} , {res['error']:7.2f} , "
            f"{res['MT/lap']:6.2f} , {res['IDe/lap']:7.2f} , {res['Be']:6.2f} , {res['IPe']:7.2f} ,"
        )
        # Handle NaN values (for unrecorded cycles)
        line = line.replace('   nan', '      -').replace('  nan', '     -')
        output.append(line)
        
    return "\n".join(output)

# Execute the main function
results_table = process_circular_task_data(DATA_FILE_NAME, TARGET_BE_VALUES, CYCLE_TIMES_REL)

print(results_table)

Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe ,
unit ,   lap ,   pixel ,   pixel ,       % ,   s/lap , bit/lap ,  double ,   bit/s ,
Theory ,  1.00 , 209.50 ,  47.00 ,    3.88 ,         ,         ,   1.00 ,         ,
Rec001 , -10.84 , 213.86 ,  91.79 ,    3.33 ,   1.84 ,    3.87 ,   1.95 ,    2.10 ,
Rec002 , -12.09 , 212.99 ,  98.14 ,    2.49 ,   1.62 ,    3.77 ,   0.73 ,    2.33 ,
Rec003 , -13.35 , 210.66 ,  60.61 ,    2.14 ,   1.47 ,    4.45 ,   0.90 ,    3.02 ,
Rec004 , -12.91 , 210.45 ,  51.96 ,    1.33 ,   1.51 ,    4.67 ,   0.79 ,    3.10 ,
Rec005 , -14.88 , 214.14 , 104.52 ,    3.56 ,   1.34 ,    3.69 ,   0.93 ,    2.75 ,
